# Introduction

This notebook demonstrates how to calibrate real and reciprocal space coordinates of scanning electron diffraction data. Calibrations include correcting the diffraction pattern for lens distortions and determining the rotation between the scan and diffraction planes based on data acquired from reference standards.

This functionaility was introduced in pyxem-0.9.0 (July 2019) and has been checked to run. Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/pyxem-demos/issues

# Contents

1. <a href='#ini'> Load Data & Initialize Generator</a>
2. <a href='#dis'> Determine Lens Distortions</a>
3. <a href='#cal'> Determine Real & Reciprocal Space Calibrations</a>
4. <a href='#rot'> Determin Real & Reciprocal Space Rotation</a> 

Import pyxem, required libraries and pyxem modules

In [ ]:
%matplotlib tk
import numpy as np
import pyxem as pxm

from pyxem.libraries.calibration_library import CalibrationDataLibrary
from pyxem.generators.calibration_generator import CalibrationGenerator

Download and the data for this demo from here and put in directory with notebooks:

https://drive.google.com/drive/folders/1YK9MZKkpuVTki9Zow5dYW9ZR1kIsbXRO?usp=sharing

# <a id='ini'></a> 1. Load Data & Initialize Generator

Load spatially averaged diffraction pattern from Au X-grating for distortion calibration

In [ ]:
au_dpeg = pxm.load_hspy('./au_xgrating_20cm.tif',
                        assign_to='electron_diffraction2d')

Load a VDF image of Au X-grating for scan pixel calibration

In [ ]:
au_im = pxm.load_hspy('./au_xgrating_100kX.hspy')

Load spatially averaged diffraction pattern from MoO3 standard for rotation calibration

In [ ]:
moo3_dpeg = pxm.load_hspy('./moo3_20cm.tif',
                          assign_to='electron_diffraction2d')

Load a VDF image of MoO3 standard for rotation calibration

In [ ]:
moo3_im = pxm.load_hspy('./moo3_100kX.tif')

Define a CalibrationDataLibary as a container for calibration data.

In [ ]:
cal_lib = CalibrationDataLibrary(au_x_grating_dp=au_dpeg,
                                 au_x_grating_im=au_im,
                                 moo3_dp=moo3_dpeg,
                                 moo3_im=moo3_im)

Plot the calibration library data for inspection

In [ ]:
cal_lib.plot_calibration_data(data_to_plot='au_x_grating_dp',
                              cmap='inferno', vmax=0.1)

In [ ]:
cal_lib.plot_calibration_data(data_to_plot='au_x_grating_im')

In [ ]:
cal_lib.plot_calibration_data(data_to_plot='moo3_dp',
                              cmap='inferno', vmax=1)

In [ ]:
cal_lib.plot_calibration_data(data_to_plot='moo3_im')

Initialise a CalibrationGenerator with the CalibrationDataLibrary

In [ ]:
cal = CalibrationGenerator(calibration_data=cal_lib)

# <a id='ids'></a> 2. Determine Lens Distortions

Lens distortions are assumed to be dominated by elliptical distortion due to the projector lens system. See, for example: https://www.sciencedirect.com/science/article/pii/S0304399105001087?via%3Dihub

Distortion correction is based on measuring the ellipticity of a ring pattern obtained from an Au X-grating calibration standard in scaninng mode.

Determine distortion correction matrix by ring fitting

In [ ]:
cal.get_elliptical_distortion(mask_radius=10,
                              scale=100, amplitude=1000,
                              asymmetry=0.9,spread=2)

Obtain residuals before and after distortion correction and plot to inspect, the aim is for any differences to be small and circularly symmetric

In [ ]:
residuals = cal.get_distortion_residuals(mask_radius=10, spread=2)
residuals.plot(cmap='RdBu', vmax=0.04)

Plot distortion corrected diffraction pattern with adjustable reference circle for inspection

In [ ]:
cal.plot_corrected_diffraction_pattern()

Check the affine matrix, which may be applied to other data

In [ ]:
cal.affine_matrix

Inspect the ring fitting parameters

In [ ]:
cal.ring_params

Calculate correction matrix and confirm that in this case it is equal to the affine matrix

In [ ]:
cal.get_correction_matrix()

# <a href='#cal'></a> 3. Determining Real & Reciprocal Space Scales

Determine the diffraction pattern calibration in reciprocal Angstroms per pixel

In [ ]:
cal.get_diffraction_calibration(mask_length=30,
                                linewidth=5)

Plot the calibrated diffraction data to check it looks about right

In [ ]:
cal.plot_calibrated_data(data_to_plot='au_x_grating_dp',
                         cmap='magma', vmax=0.1)

Plot the cross grating image data to define the line along which to take trace

In [ ]:
line = pxm.roi.Line2DROI(x1=4.83957, y1=44.4148, x2=246.46, y2=119.159, linewidth=5.57199)

cal_lib.plot_calibration_data(data_to_plot='au_x_grating_im',
                              roi=line)

Obtain the navigation calibration from the trace

In [ ]:
trace = line(cal_lib.au_x_grating_im).as_signal1D(0)
trace.plot()

In [ ]:
cal.get_navigation_calibration(line_roi=line, x1=40.,x2=232.,
                               n=3, xspace=500.)

# <a href='#rot'></a> 4. Determining Real & Reciprocal Space Rotation

MoO3 calibration data acquired. Crystal structure taken from SpringerMaterials "alpha-MoO3 (MoO3 rt) Crystal Structure" in the standard setting as:

Orthorhombic, Pnma (62), a = 13.825 A, b = 3.694 A, c = 3.954 A, alpha = 90, beta = 90, gamma = 90.

b/c ratio = 0.934

In this setting, large facets are expected to be (001) and the long axis is parallel to [010].

Note that the early published structure of MoO3 was reported in the non-standard Pbnm setting (a=3.954, b=13.825, c=3.694), which is commonly used in literature and in that case the long axis is parallel to [001]. Much of the literature describing this calibration is also inconsistent or incorrect.

In [ ]:
recip_line = pxm.roi.Line2DROI(x1=-0.30367, y1=-1.21457, x2=0.344978, y2=1.24927, linewidth=0.115582)

cal.plot_calibrated_data(data_to_plot='moo3_dp', cmap='magma', vmax=1, line=recip_line)

In [ ]:
1 / ((1.77 - 0.72)/4)

In [ ]:
real_line = pxm.roi.Line2DROI(x1=2.69824, y1=81.4867, x2=229.155, y2=61.6898, linewidth=3)

cal.plot_calibrated_data(data_to_plot='moo3_im', line=real_line)

Calculate the rotation angle between the lines

In [ ]:
cal.get_rotation_calibration(real_line, recip_line)

Determine the correction matrix including both affine distortion correction and rotation

In [ ]:
cal.get_correction_matrix()

Plot RGB overlay showing alignment of the diffraction pattern with the MoO3 particle morphology

In [ ]:
cal.plot_calibrated_data(data_to_plot='rotation_overlay')